# Micro extraction mechanism model

### Pedro S. Peixoto (ppeixoto@usp.br)

### Example of model usage 


In this example notebook, we will reproduce Figure 2 of Zhang and Pawliszyn (1993)

 - Z. Zhang, J. Pawliszyn, Headspace solid-phase microextraction, *Analytical chemistry* 65 (1993) 1843–1852.

In [4]:
#First import numpy, since all arrays are based on numpy
import numpy as np 

#Import matplotlib, for plotting
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy'

In [ ]:
#Import the main model library
import mextractmodel as mex

All parameters must be adopted in the same units of space/time/mass/volume
here we adopt:
- space units: "cm" 
- time units: "seconds" (s)
- mass: micro-grams (mug)
- volume: cm^3 (mL)


In [ ]:
#Experiment basename
name = "microextraction_zhang1993_fig2"

Now we need to set the domain, that is, the position of the interfaces.

In [ ]:
#Domain definition (position of interfaces) 
# x0   x1   x2
# |----|----|
#x=np.array([initial_position, interface_1, interface_2, final_position])
#x=np.array([0, 1]) # cm
#x=np.array([0, 0.5, 1]) # cm
x=np.array([0, 0.0056, 0.07, 0.1]) # cm Zhang 1993 Fig 2

Now we set the names of the compartments, for ploting purposes

In [ ]:
#Names of compartments
#xnames = ["", "", ""]
xnames = ["coating", "headspace", "aqueous"]

Set initial concentrations, constant in each compartment.

In [ ]:
#Initial concentrations in each space
#  Assumed constant for each space
#   C0   C1   C2
# |----|----|----|
#C=np.array([0.0, 1.0]) #mug/mL
C=np.array([0.0, 0.0, 1.0]) #mug/mL Zhang 1993 Fig 2


Diffusion coefficients for each compartment, constant at each one.

In [ ]:
#Diffusion coefficients for each space
#   D0   D1   D2
# |----|----|----|
#D=np.array([2.0, 1.0]) # 2 spaces (cm^2/s)
D=np.array([2.8e-6, 0.077, 1.8e-5]) # (cm^2/s) Zhang 1993 Fig 2

Set the partition coefficients. Always enforce zero at external boundaries for no flow condition.

In [ ]:
#Partition coefficients K for each interface
# Set 0.0 for beggining and endpoints    
# K0  K1  K2   K3
# |---|---|----|
#K=np.array([0.0, 2.0, 0.0]) # 2 boundaries and 1 interface coefficient, non-dimensional
K=np.array([0.0, 50.0, 0.2, 0.0]) # Zhang 1993 Fig 2

Now configure runtime options

In [ ]:
#Max time definition 
#maxtime = 10*60 #10 minutes
maxtime = 120 #seconds

#Time step size
dt = 0.001  #Seconds 

#Plotting time spots
iplot_time=np.array([0.0, 3.0, 15.0, 30.0, 60.0, 120])
#iplot_time=np.linspace(0, 50, 21, endpoint=True)

#Space discretization (number of grid points)
N = 200

We now build the device with these parameters (we have given the same names of the variables as the device function arguments, but we could have called the variables differently)

In [ ]:
p = mex.device(D = D, K = K, C = C, xspace = x, 
        xnames = xnames, name = name, 
        N = N, dt = dt, maxtime = maxtime, iplot_time = iplot_time)



Now that the device structure is built, we can actually use in a few ways. 

Lets loop in time the model to see how the concentrations evolve in time. The main function here is run(), which calculates all time steps of an implicit solver for the model of the device. It will save and return snapshots at all time set in iplot_time.

In [ ]:
concentrations = p.run()

The snapshots were saved in the csv files and also returned to the list of snapshots given at "concentrations". We can use this output to plot the concentrations, as follows.

In [ ]:
fig, axes = plt.subplots(1, 1, constrained_layout=True, figsize=(15,10))
plt.xlabel("x-distance (cm)")
plt.ylabel("Concentration ($\mu g/mL$)")
plt.title("Microextration Model")

for i, c in enumerate(concentrations):    
    #Plot
    t = iplot_time[i]
    axes.plot(p.x, c, label=str(t)+"s")
    
axes.legend()


The model also calculates the theoretical equilibrium state, which may be accessed via p.u_equi_ext. We can plot them all together.

In [ ]:
fig, axes = plt.subplots(1, 1, constrained_layout=True, figsize=(15,10))
plt.xlabel("x-distance (cm)")
plt.ylabel("Concentration ($\mu g/mL$)")
plt.title("Microextration Model")

for i, c in enumerate(concentrations):    
    #Plot
    t = iplot_time[i]
    axes.plot(p.x, c, label=str(t)+"s")
    
axes.plot(p.x, p.u_equi_ext, 'k--', label="Theoretical \n Equilibrium")
axes.legend()
plt.savefig(p.basename+"_final.png")

We can also add any fancy naming

In [ ]:
fig, axes = plt.subplots(1, 1, constrained_layout=True, figsize=(15,10))
plt.xlabel("x-distance (cm)")
plt.ylabel("Concentration ($\mu g/mL$)")
plt.title("Microextration Model")

for i, name in enumerate(xnames):
    x=0.8*p.xspace[i]+0.2*p.xspace[i+1]
    plt.text(x, -0.1, name)
    
for i, c in enumerate(concentrations):    
    #Plot
    t = iplot_time[i]
    axes.plot(p.x, c, label=str(t)+"s")
    
axes.plot(p.x, p.u_equi_ext, 'k--', label="Theoretical \n Equilibrium")
axes.legend()
plt.savefig(p.basename+"_final.png")